In [ ]:
import matplotlib
import argparse
# matplotlib.use('Agg')
import matplotlib.pyplot as plt
import copy
import numpy as np
from torchvision import datasets, transforms
import torch
import torch.nn as nn # import modules

from utils.sampling import mnist_iid, mnist_noniid, cifar_iid
from utils.options import args_parser
from models_v1.Update import LocalUpdate
from models_v1.Nets import MLP, CNNMnist, CNNCifar, LeNet, CNNMnist2, MobileNetV2, LeNet5, LeNet10
from models_v1.Fed import FedAvg
from models_v1.Fed import FedQAvg, Quantization, Quantization_Finite, my_score, my_score_Finite
from models_v1.test import test_img

%load_ext autoreload
%autoreload 2
from torch.autograd import grad
parser = argparse.ArgumentParser(description='Deep Leakage from Gradients.')
parser.add_argument('--index', type=int, default="25",
                    help='the index for leaking images on CIFAR.')
parser.add_argument('--image', type=str,default="",
                    help='the path to customized image.')
args = parser.parse_args([])
from torch.autograd import grad

In [ ]:
class my_argument:    
    epochs = 400    #"rounds of training"
    num_users = 5 # "number of users: K"
    frac = 0.5 #"the fraction of clients: C"
    local_ep=5 #"the number of local epochs: E"
    local_bs=10000 #"local batch size: B"
    bs=128 #"test batch size"
    lr=0.001 #"learning rate"
    momentum=0.5 # "SGD momentum (default: 0.5)"
    split='user' # "train-test split type, user or sample"
    weight_decay = 5e-4
    opt = 'ADAM'

    # model arguments
    model = 'cnn'
    kernel_num=9 #, help='number of each kind of kernel')
    kernel_sizes='3,4,5' #  help='comma-separated kernel size to use for convolution')
    norm='batch_norm' #, help="batch_norm, layer_norm, or None")
    num_filters=32 #, help="number of filters for conv nets")
    max_pool='True' #help="Whether use max pooling rather than strided convolutions")

    # other arguments
    dataset='cifar' #, help="name of dataset")
    iid=1
    num_classes=10#, help="number of classes")
    num_channels=3#, help="number of channels of imges")
    gpu=1#, help="GPU ID, -1 for CPU")
    stopping_rounds=10#, help='rounds of early stopping')
    verbose='False'#, help='verbose print')
    seed=1#, help='random seed (default: 1)')
    
args = my_argument()

args.device = torch.device('cuda:{}'.format(args.gpu) if torch.cuda.is_available() and args.gpu != -1 else 'cpu')


use_cuda = torch.cuda.is_available()
print(use_cuda)
args.device = torch.device("cpu")
args.device = torch.device("cuda:1" if use_cuda else "cpu")
print(args.device)

In [ ]:
arch = 'ConvNet64'
arch='cnn'
arch='ResNet18'
num_images = args.num_users
trained_model = False

In [ ]:
import inversefed
setup = inversefed.utils.system_startup()
defs = inversefed.training_strategy('conservative')
dm = torch.as_tensor(inversefed.consts.cifar10_mean, **setup)[:, None, None]
ds = torch.as_tensor(inversefed.consts.cifar10_std, **setup)[:, None, None]


In [ ]:
loss_fn, trainloader, validloader =  inversefed.construct_dataloaders('CIFAR10', defs)
def plot(tensor):
    tensor = tensor.clone().detach()
    tensor.mul_(ds).add_(dm).clamp_(0, 1)
    if tensor.shape[0] == 1:
        return plt.imshow(tensor[0].permute(1, 2, 0).cpu());
    else:
        fig, axes = plt.subplots(1, tensor.shape[0], figsize=(12, tensor.shape[0]*12))
        for i, im in enumerate(tensor):
            axes[i].imshow(im.permute(1, 2, 0).cpu());
num_users=args.num_users
index=[1290,1245,1110,410,1270]
index=[2661, 5723, 6633, 7758, 1245 ]
#index=[301,302,303,304,305]
index=[4434, 5723, 410, 1455,467 ]
ground_truth, labels = [], []
ground_truth_joint,labels_joint=[],[]
for i in range(num_users):
    ground_truth.append([])
    labels.append([])
idx = 0 # choosen randomly ... just whatever you want
    #while len(labels) < num_images:
index=[1290, 410, 303, 1245, 5585 ]
index=[1290,1245,1110,410,1270]
#index=[5723, 410, 2661, 1245, 5585 ]
index=[5723, 410, 2661, 1245, 5585 ]
index= [305, 410, 4434, 1455, 8034]
index=[301, 302,303, 304, 305]
index=[303, 305, 4434, 410, 5723]
index=[4434, 5723, 410, 1455,467 ] #This one is also in the list
index=[1567, 3149, 3532, 1694, 770]
index=[4434, 3149, 3532, 410,1455]
for i in range(num_users):
    print(i)
    img, label = validloader.dataset[index[idx]]
    idx += 1
    labels[i].append(torch.as_tensor((label,), device=setup['device']))
    ground_truth[i].append(img.to(**setup))
    ground_truth[i] = torch.stack(ground_truth[i])
    ground_truth_joint.append(img.to(**setup))
    
    labels[i] = torch.cat(labels[i])
    labels_joint.append(torch.as_tensor((label,), device=setup['device']))
    plot(ground_truth[i]);
    #plt.show()
    print([validloader.dataset.classes[l] for l in labels[i]]);


# In[35]:


ground_truth_joint = torch.stack(ground_truth_joint)
labels_joint = torch.cat(labels_joint)


# In[36]:


plot(ground_truth_joint);
plt.savefig("original2.png")
print([validloader.dataset.classes[l] for l in labels_joint]);


In [ ]:

local_lr = 1e-4
local_steps = 1
use_updates = True

In [ ]:
A=torch.load("Model.pt")
print(A)
net_glob, _ = inversefed.construct_model(arch, num_classes=10, num_channels=3)
#print(net_glob.state_dict())


In [ ]:
net_glob.to(**setup)
#print(net_glob.state_dict())
#net_glob.zero_grad()
#print(net_glob.state_dict())
target_loss, _, _ = loss_fn(net_glob(ground_truth[0]), labels[0])
input_gradient=inversefed.reconstruction_algorithms.loss_steps(net_glob, ground_truth[0], labels[0], 
                                                        lr=local_lr, local_steps=local_steps,
                                                                   use_updates=use_updates)
net_glob.load_state_dict(A)
w=net_glob.state_dict()
net_glob.zero_grad()
net_glob.zero_grad()
#print(w)

In [ ]:
u1=np.load("gradient11.npy")
u2=np.load("gradient22.npy")
u3=np.load("gradient33.npy")
u4=np.load("gradient44.npy")
u5=np.load("gradient55.npy")
u6=np.load("gradient66.npy")
user_gradient=[]
for i in range(args.num_users):
    user_gradient.append([])
for i in range(args.num_users):
    user_gradient[i]=np.concatenate((u1[i],u2[i],u3[i], u4[i], u5[i], u6[i]),axis=0)
backup_gradient=user_gradient
print(len(user_gradient[2]))

In [ ]:
local_lr = 1e-4
local_steps = 1
use_updates = True

In [ ]:
original_gradient=np.load("original_grad.npy")

In [ ]:
#sum_FedAvg_tensor=input_gradient[0]
flat=[]
count=0
u=0
#user_gradient=[[]]*args.num_users
for user in range(args.num_users):
    #user_gradient[user]=backup_gradient[user]
    user_gradient[user]=backup_gradient[user]
print(backup_gradient[1][0:10])
print(user_gradient[1][0:10])
user_gradient_tensor=[]
#for user in idxs_users:
#user_gradient[u]=o[u]
print(len(user_gradient[u]))
user_gradient_tensor=input_gradient

flat=[]
count=0
print(user_gradient[1][0:10])
for j in range(len(user_gradient_tensor)): # 4 layers in parameter
    flat.append([])
    #for h in range(len(model_diff[user])):
for h in range(len(user_gradient_tensor)):
    #print(user_gradient[u])
#for h in user_gradient_tensor.keys():
    s=list(user_gradient_tensor[h].shape)
        #print(s)
    if (len(s)==0):
        new=np.array(0)
        user_gradient[u]=np.delete(user_gradient[u],np.s_[0])
    else:
        #print(user_gradient[u])
        z=np.prod(list(user_gradient_tensor[h].shape))
        flat[count]=user_gradient[u][0:z] # taking out the vector for the specified layer
        user_gradient[u]=np.delete(user_gradient[u],np.s_[0:z])# deleting that vector from decoded after taking out
            
        new=np.array(flat[count]).reshape(list(user_gradient_tensor[h].shape)) #reshaping back to the marix
              
    user_gradient_tensor[h]=torch.from_numpy(new).to(torch.device("cuda")) #converting the matrix to a tensor
            #print(w_glob[cluster_no][h].shape)
    count=count+1

In [ ]:
flat=[]
count=0

# user_gradient_tensor=[]
# #for user in idxs_users:
# for user in range(0,num_users):
#     #user_gradient_tensor.append(model_diff[user])
#     #user_gradient_tensor=model_diff[user]
#     #print(user_gradient_tensor)
#     user_gradient[user]=backup_gradient[user]
#     user_gradient_tensor.append(input_gradient[0])

#     flat=[]
#     count=0
#     for j in range(len(input_gradient[user])): # 4 layers in parameter
#         flat.append([])
#     #for h in range(len(model_diff[user])):
#     for h in range(len(user_gradient_tensor[user])):
#         s=list(user_gradient_tensor[user][h].shape)
#         if (len(s)==0):
#            new=np.array(0)
#            user_gradient[user]=np.delete(user_gradient[user],np.s_[0])
#         else:
#            z=np.prod(list(user_gradient_tensor[user][h].shape))
#            flat[count]=user_gradient[user][0:z] # taking out the vector for the specified layer
#            user_gradient[user]=np.delete(user_gradient[user],np.s_[0:z])# deleting that vector from decoded after taking out
            
#            new=np.array(flat[count]).reshape(list(user_gradient_tensor[user][h].shape)) #reshaping back to the marix
              
#         user_gradient_tensor[user][h]=torch.from_numpy(new).to(torch.device("cuda")) #converting the matrix to a tensor
#             #print(w_glob[cluster_no][h].shape)
#         count=count+1



#This has been taken from Geiping's paper's code for gradient inversion available at https://github.com/JonasGeiping/invertinggradients

config = dict(signed=True,
              boxed=True,
              cost_fn='sim',
              indices='def',
              weights='equal',
              lr=0.01,
              optim='adam',
              restarts=8,
              max_iterations=24000,
              total_variation=1e-6,
              init='randn',
              filter='none',
              lr_decay=True,
              scoring_choice='loss')

#rec_machine = inversefed.GradientReconstructor(model, (dm, ds), config, num_images=num_images)
#rec_machine =  inversefed.FedAvgReconstructor(model, (dm, ds), local_steps, local_lr, config,
                                             #use_updates=use_updates, num_images=num_images)
for user in range(0,1):
     rec_machine =  inversefed.FedAvgReconstructor(net_glob, (dm, ds), local_steps, local_lr, config,
                                             use_updates=use_updates, num_images=1)
     output, stats = rec_machine.reconstruct(user_gradient_tensor, labels[u], img_shape=(3, 32, 32))

     test_mse = (output.detach() - ground_truth[u]).pow(2).mean()
     feat_mse = (net_glob(output.detach())- net_glob(ground_truth[u])).pow(2).mean()  
     test_psnr = inversefed.metrics.psnr(output, ground_truth[u], factor=1/ds)

     plot(output)
     plt.title(f"Rec. loss: {stats['opt']:2.4f} | MSE: {test_mse:2.4f} "
           f"| PSNR: {test_psnr:4.2f} | FMSE: {feat_mse:2.4e} |");
     plt.savefig("recovered2.png")

In [ ]:
print(type(output))
#print(output)
torch.save(output, "frog_blue_topK_new_1000.pt")

In [ ]:
output2=torch.load("frog_blue_topK_new_1000.pt")
plot(output2)
test_mse = (output2.detach() - ground_truth[u]).pow(2).mean()
print(test_mse)
feat_mse = (net_glob(output2.detach())- net_glob(ground_truth[u])).pow(2).mean()  
test_psnr = inversefed.metrics.psnr(output2, ground_truth[u], factor=1/ds)
print(feat_mse)
print(test_psnr)

In [ ]:
u=4
output2=torch.load("output_0.pt")
#plot(output2)
output2=output2[0]
test_mse = (output2.detach() - ground_truth[u]).pow(2).mean()
print(test_mse)
# feat_mse = (net_glob(output2.detach())- net_glob(ground_truth[u])).pow(2).mean()  
# test_psnr = inversefed.metrics.psnr(output2, ground_truth[u], factor=1/ds)
# print(feat_mse)
# print(test_psnr)

In [ ]:
plot(output2)
torch.save(output2, "bird_randK_400.pt")

In [ ]:
output2=torch.load("horse_topK2.pt")
plot(output2)
test_mse = (output2.detach() - ground_truth[u]).pow(2).mean()
print(test_mse)
print(u)

In [ ]:
plt.title(f"Rec. loss: {stats['opt']:2.4f} | MSE: {test_mse:2.4f} "
#           f"| PSNR: {test_psnr:4.2f} | FMSE: {feat_mse:2.4e} |");

In [ ]:
print(test_psnr)
print(test_mse)

In [ ]:
print(len(input_gradient))
print(flat[count])
print(backup_gradient[u])

In [ ]:
print(count)
print(s)

In [ ]:
print(backup_gradient[u])